# Mercury DS Manager Take-Home

Exploring customer onboarding and product adoption data. Looking at 3 datasets - organizations, funnel stages, and product usage.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
orgs = pd.read_csv('organizations.csv')
orgs.head()

,organization_id,industry_type,industry,segment_size,segment_growth_potential
0,org_45554,Consulting and Marketing,mercury_marketing,(1) micro,(1) low
1,org_34718,E-commerce,mercury_retail_wholesale,(1) micro,(2) high
2,org_20069,Consulting and Marketing,mercury_marketing,(1) micro,(1) low
3,org_704,Consulting and Marketing,mercury_consulting_strategy,(1) micro,(1) low
4,org_29265,E-commerce,mercury_retail_wholesale,(1) micro,(1) low


In [3]:
orgs.shape

(500, 5)

500 orgs

In [4]:
orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   organization_id           500 non-null    object
 1   industry_type             500 non-null    object
 2   industry                  500 non-null    object
 3   segment_size              500 non-null    object
 4   segment_growth_potential  500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [5]:
orgs.isnull().sum()

organization_id             0
industry_type               0
industry                    0
segment_size                0
segment_growth_potential    0
dtype: int64

No nulls

In [6]:
orgs['industry_type'].value_counts()

industry_type
E-commerce                  223
Technology                  153
Consulting and Marketing    124
Name: count, dtype: int64

3 main industry types. E-commerce biggest (223), then Tech (153)

In [7]:
orgs['segment_size'].value_counts()

segment_size
(1) micro     479
(2) small      13
(3) medium      8
Name: count, dtype: int64

Mostly micro orgs. Small sample sizes for small/medium

In [8]:
# Clean up formatting
orgs['segment_size'] = orgs['segment_size'].str.replace(r'\(\d+\)\s*', '', regex=True)
orgs['segment_growth_potential'] = orgs['segment_growth_potential'].str.replace(r'\(\d+\)\s*', '', regex=True)
orgs['segment_size'].value_counts()

segment_size
micro     479
small      13
medium      8
Name: count, dtype: int64

In [9]:
pd.crosstab(orgs['segment_size'], orgs['segment_growth_potential'])

segment_growth_potential,high,low
segment_size,,
medium,3,5
micro,143,336
small,9,4


Most analysis will need to focus on industry_type since segment sizes are too small

In [10]:
funnel = pd.read_csv('adoption_funnel.csv')
funnel.head()

,organization_id,funnel_stage,date
0,org_45554,application_submitted,2024-09-12
1,org_34718,application_submitted,2024-07-17
2,org_20069,application_submitted,2024-04-25
3,org_704,application_submitted,2024-01-06
4,org_29265,application_submitted,2024-06-15


In [11]:
funnel.shape

(2000, 3)

2000 rows for 500 orgs = 4 stages per org

In [12]:
funnel['funnel_stage'].value_counts()

funnel_stage
application_submitted    500
approved                 500
first_deposit            500
first_active             500
Name: count, dtype: int64

All 500 orgs have all 4 stages

In [13]:
funnel.isnull().sum()

organization_id      0
funnel_stage         0
date               790
dtype: int64

790 null dates. Not everyone completes all stages

In [14]:
funnel.groupby('funnel_stage')['date'].apply(lambda x: x.isnull().sum())

funnel_stage
application_submitted      0
approved                 222
first_active             303
first_deposit            265
Name: date, dtype: int64

In [15]:
# How many got approved?
funnel[(funnel['funnel_stage'] == 'approved') & (funnel['date'].notna())].shape[0]

278

278 got approved out of 500

55.6% approval rate

In [16]:
# Activation rate?
funnel[(funnel['funnel_stage'] == 'first_active') & (funnel['date'].notna())].shape[0]

197

In [17]:
197 / 500

0.394

39.4% activation rate. Big drop from 55.6% approval to activation

In [18]:
# Does approval rate differ by industry?
approved_orgs = funnel[(funnel['funnel_stage'] == 'approved') & (funnel['date'].notna())]['organization_id'].unique()
orgs['got_approved'] = orgs['organization_id'].isin(approved_orgs)
orgs.groupby('industry_type')['got_approved'].mean()

industry_type
Consulting and Marketing    0.572581
E-commerce                  0.452915
Technology                  0.692810
Name: got_approved, dtype: float64

Tech 69%, E-commerce 45%, Consulting 57%. Big differences

In [19]:
products = pd.read_csv('product_usage.csv')
products.head()

,organization_id,day,product,is_active
0,org_45554,2024-09-25,Bank Account,False
1,org_34718,2024-11-13,Invoicing,False
2,org_20069,2024-11-19,Invoicing,False
3,org_704,2024-09-13,Invoicing,False
4,org_29265,2024-12-27,Bank Account,False


In [20]:
products.shape

(200480, 4)

In [21]:
products['organization_id'].nunique()

278

278 unique orgs. Same as approved count!

In [22]:
products['product'].value_counts()

product
Bank Account    50120
Invoicing       50120
Credit Card     50120
Debit Card      50120
Name: count, dtype: int64

In [23]:
products['is_active'].value_counts()

is_active
False    174972
True      25508
Name: count, dtype: int64

Mostly inactive. 25k active vs 175k inactive records

In [24]:
# Which products are most active?
products[products['is_active'] == True]['product'].value_counts()

product
Debit Card      17120
Bank Account     5591
Credit Card      2749
Invoicing          48
Name: count, dtype: int64

Debit Card most active (17k), then Bank Account (5.6k). Invoicing barely used (48)

In [25]:
# Do different industries use different products?
active_products = products[products['is_active'] == True]
product_users = active_products.groupby(['organization_id', 'product']).size().reset_index(name='active_days')
product_users = product_users.merge(orgs[['organization_id', 'industry_type']], on='organization_id')
product_users.head()

,organization_id,product,active_days,industry_type
0,org_1042,Bank Account,188,Technology
1,org_1042,Debit Card,324,Technology
2,org_1042,Invoicing,11,Technology
3,org_10702,Bank Account,143,Consulting and Marketing
4,org_10702,Debit Card,292,Consulting and Marketing


In [26]:
# Product adoption by industry
adoption_counts = product_users.groupby(['industry_type', 'product']).size().reset_index(name='adopters')
industry_totals = orgs[orgs['got_approved'] == True].groupby('industry_type').size().reset_index(name='total_approved')
adoption_rates = adoption_counts.merge(industry_totals, on='industry_type')
adoption_rates['adoption_rate'] = adoption_rates['adopters'] / adoption_rates['total_approved'] * 100
adoption_rates.pivot(index='industry_type', columns='product', values='adoption_rate').fillna(0).round(1)

product,Bank Account,Credit Card,Debit Card,Invoicing
industry_type,,,,
Consulting and Marketing,63.4,4.2,49.3,7.0
E-commerce,57.4,3.0,48.5,1.0
Technology,66.0,13.2,49.1,9.4


Tech way higher on Credit Card (13% vs 3%). Clear industry preferences

In [27]:
# Is this significant?
from scipy.stats import chi2_contingency

# Tech vs others for Credit Card
tech_cc = product_users[(product_users['industry_type'] == 'Technology') & (product_users['product'] == 'Credit Card')].shape[0]
tech_total = orgs[(orgs['industry_type'] == 'Technology') & (orgs['got_approved'] == True)].shape[0]
other_cc = product_users[(product_users['industry_type'] != 'Technology') & (product_users['product'] == 'Credit Card')].shape[0]
other_total = orgs[(orgs['industry_type'] != 'Technology') & (orgs['got_approved'] == True)].shape[0]

contingency = [[tech_cc, tech_total - tech_cc], [other_cc, other_total - other_cc]]
chi2, p_value = chi2_contingency(contingency)[:2]
print(f'p-value: {p_value:.3f}')

p-value: 0.005


p = 0.005. Significant

In [28]:
# What about high vs low growth segments?
product_users_growth = product_users.merge(orgs[['organization_id', 'segment_growth_potential']], on='organization_id')
growth_adoption = product_users_growth.groupby(['segment_growth_potential', 'product']).size().reset_index(name='adopters')
growth_totals = orgs[orgs['got_approved'] == True].groupby('segment_growth_potential').size().reset_index(name='total_approved')
growth_rates = growth_adoption.merge(growth_totals, on='segment_growth_potential')
growth_rates['adoption_rate'] = growth_rates['adopters'] / growth_rates['total_approved'] * 100
growth_rates.pivot(index='segment_growth_potential', columns='product', values='adoption_rate').fillna(0).round(1)

product,Bank Account,Credit Card,Debit Card,Invoicing
segment_growth_potential,,,,
high,66.1,14.4,51.7,9.3
low,59.4,1.9,46.9,3.1


High-growth orgs adopt Credit Card 7x more (14% vs 2%)

In [ ]:
# How long does activation take?
funnel['date'] = pd.to_datetime(funnel['date'])
approved_dates = funnel[funnel['funnel_stage'] == 'approved'][['organization_id', 'date']].rename(columns={'date': 'approved_date'})
active_dates = funnel[funnel['funnel_stage'] == 'first_active'][['organization_id', 'date']].rename(columns={'date': 'active_date'})
activation_time = approved_dates.merge(active_dates, on='organization_id', how='inner')
activation_time['days_to_activate'] = (activation_time['active_date'] - activation_time['approved_date']).dt.days


print(activation_time['days_to_activate'].median())

18.0


Median 18 days to activate

In [30]:
# By industry?
activation_with_industry = activation_time.merge(orgs[['organization_id', 'industry_type']], on='organization_id')
activation_with_industry.groupby('industry_type')['days_to_activate'].median()

industry_type
Consulting and Marketing    19.0
E-commerce                  28.0
Technology                  11.0
Name: days_to_activate, dtype: float64

Tech activates faster (11 days vs 28 for E-commerce)

In [31]:
# Product churn - who stops using products?
products['day'] = pd.to_datetime(products['day'])
latest_date = products['day'].max()
last_active = products[products['is_active'] == True].groupby(['organization_id', 'product'])['day'].max().reset_index()
last_active['days_since_active'] = (latest_date - last_active['day']).dt.days
last_active['churned'] = last_active['days_since_active'] > 30
churn_by_product = last_active.groupby('product')['churned'].agg(['count', 'sum']).reset_index()
churn_by_product['churn_rate'] = churn_by_product['sum'] / churn_by_product['count'] * 100
churn_by_product

,product,count,sum,churn_rate
0,Bank Account,173,42,24.277457
1,Credit Card,20,5,25.000000
2,Debit Card,136,27,19.852941
3,Invoicing,16,7,43.750000


Churn is really high. Bank Account 24%, Invoicing 44%

## Key Findings

**Industry differences are significant:**
- Tech has 69% approval vs E-commerce 45%
- Tech adopts Credit Card 4x more (13% vs 3%)
- Tech activates faster (11 vs 28 days)

**Growth segment matters:**
- High-growth orgs adopt Credit Card 7x more (14% vs 2%)

**Product insights:**
- Debit Card most used overall (17k active records)
- Invoicing has low adoption (16 orgs) and high churn (44%)
- Big drop from approval (55.6%) to activation (39.4%)

**Experiment opportunity:**
Industry-specific product recommendations could work - clear preferences exist

## Experiment Design

**Hypothesis:** Featuring products by industry_type during onboarding increases adoption

**Treatment logic:**
- Technology → Feature Credit Card
- E-commerce → Feature Debit Card  
- Consulting → Feature Invoicing

**Why industry_type vs specific industry:** Better sample sizes (100+ vs 10-20 per group)

**Sample size:** Need ~600 approved orgs for 80% power, 20% relative lift

In [32]:
# Sample size calculation
from scipy import stats

# Current adoption rate (any non-Bank product)
current_adopters = product_users[product_users['product'] != 'Bank Account']['organization_id'].nunique()
total_approved = orgs['got_approved'].sum()
baseline_rate = current_adopters / total_approved
target_rate = baseline_rate * 1.2  # 20% relative lift

print(f'Baseline adoption: {baseline_rate:.1%}')
print(f'Target adoption: {target_rate:.1%}')
print(f'Absolute lift: {target_rate - baseline_rate:.1%}')

# Sample size for 80% power
alpha = 0.05
power = 0.8
effect_size = target_rate - baseline_rate
pooled_p = (baseline_rate + target_rate) / 2

z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta = stats.norm.ppf(power)

n_per_group = ((z_alpha + z_beta)**2 * 2 * pooled_p * (1 - pooled_p)) / effect_size**2
print(f'Need {n_per_group:.0f} per group = {n_per_group * 2:.0f} total approved orgs')

Baseline adoption: 51.8%
Target adoption: 62.2%
Absolute lift: 10.4%
Need 359 per group = 717 total approved orgs


**Metrics:**
- Primary: 30-day product adoption rate
- Guardrails: Approval rate, activation rate, churn

**Decision framework:**
- +20% lift → Full rollout
- Mixed by industry → Selective rollout
- <10% lift → Don't implement
- Guardrail failure → Stop immediately

**Runtime:** ~6 months to get 600 approved orgs